In [1]:
import sys
sys.path.insert(0,'..')

In [2]:
import torch
import os

from train import train
import priors
import utils

import numpy as np

from datasets import load_openml_list, valid_dids_classification, test_dids_classification
from tabular import evaluate, get_model, get_default_spec
from tabular import bayes_net_metric, gp_metric, knn_metric, catboost_metric, xgb_metric, logistic_metric

/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
%load_ext autoreload

%autoreload 2

## Loading Datasets

In [4]:
### Loads small list of datasets
print('Loading test datasets...')
test_datasets, test_datasets_df = load_openml_list(test_dids_classification[0:2], filter_for_nan=True)
ds = test_datasets
print('\n Loading valid datasets...')
valid_datasets, valid_datasets_df = load_openml_list(valid_dids_classification[0:2], filter_for_nan=True)

Loading test datasets...
wine 973
covertype 1596

 Loading valid datasets...
ionosphere 59


In [ ]:
### Loads all datasets
print('Loading test datasets...')
test_datasets, test_datasets_df = load_openml_list(test_dids_classification, filter_for_nan=True)
ds = test_datasets

print('\n Loading valid datasets...')
valid_datasets, valid_datasets_df = load_openml_list(valid_dids_classification, filter_for_nan=True)

## Setting params

In [5]:
device = 'cpu'

In [6]:
# After how many training samples should evaluatuion be done?
# Trained models have not been trained to evaluate after 30 samples
# so performance will drop
eval_positions = [30]

# What is the maximum number of features?
# Pretrained models have to use 60
max_features = 60

# How many samples should be loaded for one dataset?
# Samples after the training sequence are used for evaluation
seq_len = 100

# How many subsamples of datasets should be drawn for each dataset
max_samples = 20

In [7]:
gp_model_checkpoint_dir = "../results/tabular_model_gp.ckpt"
gp_model_config = {'batch_size': 512,
 'bptt': 100,
 'dropout': 0.5,
 'emsize': 512,
 'epochs': 100,
 'eval_positions': [10, 20, 40, 80],
 'lr': 6.271726842985807e-05,
 'nhead': 4,
 'nhid_factor': 2,
 'nlayers': 5,
 'num_features': 60,
 'prior_lengthscale': 0.00014803074521613278,
 'prior_noise': 0.001,
 'prior_normalize_by_used_features': True,
 'prior_num_features_used_sampler': {'uniform_int_sampler_f(1,max_features)': '<function <lambda>.<locals>.<lambda> at 0x7f21e832e550>'},
 'prior_order_y': False,
 'prior_outputscale': 2.3163584733185836,
 'prior_type': 'gp'}

In [9]:
bnn_model_checkpoint_dir = "../results/tabular_model_bnn.ckpt"
bnn_model_config = {'batch_size': 512,
 'bptt': 50,
 'dropout': 0.5,
 'emsize': 512,
 'epochs': 100,
 'eval_positions': [10, 20, 40],
 'lr': 1.6421403128751275e-05,
 'nhead': 4,
 'nhid_factor': 2,
 'nlayers': 5,
 'num_features': 60,
 'prior_activations': "<class 'torch.nn.modules.activation.Tanh'>",
 'prior_dropout_sampler': {'lambda: 0.0': '<function <lambda> at 0x7f613c1364c0>'},
 'prior_emsize_sampler': {'scaled_beta_sampler_f(2.0, 4.0, 150, 2)': '<function <lambda>.<locals>.<lambda> at 0x7f613c136310>'},
 'prior_is_causal': False,
 'prior_nlayers_sampler': {'lambda: 3': '<function <lambda> at 0x7f613c136790>'},
 'prior_noise_std_gamma_k': 1.8663049257557085,
 'prior_noise_std_gamma_theta': 0.05275478076173361,
 'prior_normalize_by_used_features': False,
 'prior_num_features_used_sampler': {'scaled_beta_sampler_f(1.0, 1.6, max_features, 2)': '<function <lambda>.<locals>.<lambda> at 0x7f613c136550>'},
 'prior_order_y': True,
 'prior_sigma_gamma_k': 3.6187797729244253,
 'prior_sigma_gamma_theta': 0.06773738681062867,
 'prior_type': 'mlp'}

## Loading PFN

In [16]:
model_type = 'bnn'
if model_type == 'gp':
    raise Exception("Not Implemented")
    config = gp_model_config
    checkpoint_dir = gp_model_checkpoint_dir
elif model_type == 'bnn':
    config = bnn_model_config
    checkpoint_dir = bnn_model_checkpoint_dir

model = get_model(config, device, eval_positions, should_train=False) # simply get Transformer model
model_state, _ = torch.load(checkpoint_dir) 
model[2].load_state_dict(model_state) # load state dict for the model
model = model[2]

Using cpu:0 device
DataLoader.__dict__ {'num_steps': 100, 'fuse_x_y': False, 'get_batch_kwargs': {'batch_size': 512, 'seq_len': 50, 'num_features': 60, 'hyperparameters': ('<function <lambda> at 0x7f613c136790>', '<function <lambda>.<locals>.<lambda> at 0x7f613c136310>', "<class 'torch.nn.modules.activation.Tanh'>", <function <lambda>.<locals>.<lambda> at 0x7f2ed182ba60>, <function <lambda>.<locals>.<lambda> at 0x7f2ff03b7040>, '<function <lambda> at 0x7f613c1364c0>', True, '<function <lambda>.<locals>.<lambda> at 0x7f613c136550>', None, False, None, None, None, True, False, None, 0.0), 'batch_size_per_gp_sample': 8}, 'num_features': 60, 'num_outputs': 1}


## Evaluation of PFN and Baselines on all datasets

### Transformer

In [11]:
device = 'cpu'
result = evaluate(ds, model.to(device), 'transformer'
                  , max_features = max_features
                  , bptt=seq_len
                  , eval_position_range=eval_positions
                  , device=device
                  , max_samples=20
                  , rescale_features=config["prior_normalize_by_used_features"]
                  , extend_features=True, plot=False, overwrite=True, save=False)
result

Evaluating wine
	 Eval position 30 done..
Evaluating covertype
	 Eval position 30 done..


{'metric': 'auc',
 'wine_mean_metric_at_30': 0.9587367346938775,
 'wine_time': 1.9455327987670898,
 'covertype_mean_metric_at_30': 0.9624857142857144,
 'covertype_time': 2.1963138580322266,
 'mean_metric_at_30': 0.960611224489796,
 'mean_metric': 0.960611224489796}

### KNN

In [11]:
result = evaluate(ds, knn_metric, 'knn'
                  , bptt=seq_len
                  , eval_position_range=eval_positions
                  , device=device
                  , max_samples=20
                  , overwrite=True
                  , save=False)
result

Evaluating wine


  0%|                                                                                                              | 0/20 [00:00<?, ?it/s]/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default

 10%|██████████▏                                                                                           | 2/20 [00:00<00:01, 14.78it/s]/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default

/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer

 20%|████████████████████▍                                                                                 | 4/20 [00:00<00:01, 15.12it/s]/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default

...

/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer

/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer

/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer

	 Eval position 30 done..


{'metric': 'auc',
 'wine_mean_metric_at_30': 0.9002846938775511,
 'wine_time': 0.8868608474731445,
 'covertype_mean_metric_at_30': 0.7934336734693876,
 'covertype_time': 0.74686598777771,
 'mean_metric_at_30': 0.8468591836734694,
 'mean_metric': 0.8468591836734694}

### Logistic Regression

In [12]:
result = evaluate(ds, logistic_metric, 'logistic'
                  , bptt=seq_len
                  , eval_position_range=eval_positions
                  , device=device
                  , max_samples=20
                  , overwrite=True
                  , save=False)
result

Evaluating wine


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:19<00:00,  1.01it/s]


	 Eval position 30 done..
Evaluating covertype


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:34<00:00,  1.75s/it]

	 Eval position 30 done..


{'metric': 'auc',
 'wine_mean_metric_at_30': 0.9295571428571429,
 'wine_time': 19.71760869026184,
 'covertype_mean_metric_at_30': 0.9083142857142859,
 'covertype_time': 34.93118238449097,
 'mean_metric_at_30': 0.9189357142857144,
 'mean_metric': 0.9189357142857144}

### Gaussian Process

In [13]:
result = evaluate(ds, gp_metric, 'gp'
                  , bptt=seq_len
                  , eval_position_range=eval_positions
                  , device=device
                  , max_samples=20
                  , overwrite=True
                  , save=False)
result

Evaluating wine


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:48<00:00,  2.41s/it]


	 Eval position 30 done..
Evaluating covertype


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:58<00:00,  2.90s/it]

	 Eval position 30 done..


{'metric': 'auc',
 'wine_mean_metric_at_30': 0.7852867346938774,
 'wine_time': 48.288535356521606,
 'covertype_mean_metric_at_30': 0.9294663265306123,
 'covertype_time': 58.08171844482422,
 'mean_metric_at_30': 0.857376530612245,
 'mean_metric': 0.857376530612245}

### XG Boost

In [14]:
result = evaluate(ds, xgb_metric, 'xgb'
                  , bptt=seq_len
                  , eval_position_range=eval_positions
                  , device=device
                  , max_samples=20
                  , overwrite=True
                  , save=False)
result

Evaluating wine



  0%|                                                                                                              | 0/20 [00:00<?, ?it/s]

Fitting 5 folds for each of 16 candidates, totalling 80 fits


/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/ypq/anaconda3/envs/GD/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future versio

{'colsample_bytree': 0.8, 'eval_metric': 'logloss', 'learning_rate': 0.02, 'max_depth': 1, 'min_child_weight': 0.5, 'n_estimators': 100, 'subsample': 0.5}
Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.02, max_depth=1, min_child_weight=0.5, n_estimators=100, subsample=0.5; total time=   8.9s
[CV] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.02, max_depth=1, min_child_weight=0.5, n_estimators=100, subsample=0.5; total time=   3.0s
[CV] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.02, max_depth=1, min_child_weight=0.5, n_estimators=100, subsample=0.8; total time=   2.6s
[CV] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.02, max_depth=1, min_child_weight=1.0, n_estimators=100, subsample=0.5; total time=   1.0s
[CV] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.02, max_depth=1, min_child_weight=1.0, n_estimators=100, subsample=0.8; total ti

...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [19:23<00:00, 58.19s/it]

{'colsample_bytree': 0.8, 'eval_metric': 'logloss', 'learning_rate': 0.02, 'max_depth': 1, 'min_child_weight': 0.5, 'n_estimators': 100, 'subsample': 0.8}
	 Eval position 30 done..


{'metric': 'auc',
 'wine_mean_metric_at_30': 0.9325163265306122,
 'wine_time': 1184.659984588623,
 'covertype_mean_metric_at_30': 0.7929112244897958,
 'covertype_time': 1163.8565165996552,
 'mean_metric_at_30': 0.862713775510204,
 'mean_metric': 0.862713775510204}

[CV] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.02, max_depth=1, min_child_weight=0.5, n_estimators=100, subsample=0.8; total time=   3.9s
[CV] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.02, max_depth=1, min_child_weight=1.0, n_estimators=100, subsample=0.5; total time=   3.1s
[CV] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.02, max_depth=2, min_child_weight=0.5, n_estimators=100, subsample=0.5; total time=   4.5s
[CV] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.02, max_depth=2, min_child_weight=0.5, n_estimators=100, subsample=0.8; total time=   5.9s
[CV] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.02, max_depth=2, min_child_weight=1.0, n_estimators=100, subsample=0.8; total time=   6.1s
[CV] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.2, max_depth=1, min_child_weight=0.5, n_estimators=100, subsample=0.5; total time=   3.1s
[CV] END colsample_bytree=0.8, eval_metric=logl

[CV] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.02, max_depth=1, min_child_weight=0.5, n_estimators=100, subsample=0.8; total time=   2.9s
[CV] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.02, max_depth=1, min_child_weight=1.0, n_estimators=100, subsample=0.5; total time=   1.4s
[CV] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.02, max_depth=1, min_child_weight=1.0, n_estimators=100, subsample=0.5; total time=   2.8s
[CV] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.02, max_depth=1, min_child_weight=1.0, n_estimators=100, subsample=0.8; total time=   3.3s
[CV] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.02, max_depth=2, min_child_weight=0.5, n_estimators=100, subsample=0.5; total time=   2.9s
[CV] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.02, max_depth=2, min_child_weight=1.0, n_estimators=100, subsample=0.5; total time=   3.6s
[CV] END colsample_bytree=0.8, eval_metric=log

### Bayesian NN

In [15]:
result = evaluate(ds, bayes_net_metric, 'bayes_net'
                  , bptt=seq_len
                  , eval_position_range=eval_positions
                  , device=device
                  , max_samples=20
                  , overwrite=True
                  , save=False)
result

Evaluating wine


  0%|                                                                                                              | 0/20 [00:00<?, ?it/s]


ValueError: Invalid parameter num_samples_for_prediction for estimator BayesianNNClassifier(device=device(type='cpu'), embed=5, lr=0.001, n_layers=2,
                     num_features=13). Check the list of available parameters with `estimator.get_params().keys()`.

### Catboost

In [16]:
result = evaluate(ds, catboost_metric, 'catboost'
                  , bptt=seq_len
                  , eval_position_range=eval_positions
                  , device=device
                  , max_samples=20
                  , overwrite=True
                  , save=False)
result

Evaluating wine


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [06:47<00:00, 20.38s/it]


	 Eval position 30 done..
Evaluating covertype


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [06:23<00:00, 19.17s/it]

	 Eval position 30 done..


{'metric': 'auc',
 'wine_mean_metric_at_30': 0.9076479591836735,
 'wine_time': 407.7067492008209,
 'covertype_mean_metric_at_30': 0.8101428571428572,
 'covertype_time': 383.377445936203,
 'mean_metric_at_30': 0.8588954081632654,
 'mean_metric': 0.8588954081632654}